
Загружаем исходные данные в colab.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset(1).csv to dataset(1).csv


Загружаем данные в датафрейм

In [ ]:
import io
import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))

df

,event_date,app_version,country_code,install_date,media_source,campaign_name,appsflyer_id,is_organic,payments,revenue_usd,ad_revenue_usd,sessions,playtime
0,2022-04-09,0.1.75.1,SG,2022-04-09,bytedanceglobal_int,EM_AND_SEA_Purchase_IT2_0604,1649548002569-6720420720726321605,False,0,0.0,0.000000,1,4
1,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649545238733-4551347935758619924,False,0,0.0,0.000000,1,2
2,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649540322226-5366969563919899948,False,0,0.0,0.000000,1,2
3,2022-04-09,0.1.75.1,UK,2022-04-07,bytedanceglobal_int,EM_AND_T1_Purchase_IT2_0604,1649337475698-617904100123526593,False,0,0.0,0.000000,1,1
4,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649544985557-9160306254813664580,False,0,0.0,0.000000,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
690370,2022-07-31,NaN,PH,2022-07-28,organic,organic,1659003289354-9130797695840500796,True,0,0.0,0.002614,1,1
690371,2022-07-31,NaN,AE,2022-07-28,organic,organic,1659020308480-7169513513931602186,True,0,0.0,0.012164,1,1
690372,2022-07-31,NaN,DZ,2022-07-29,organic,organic,1659059933950-196412188438371439,True,0,0.0,0.000195,1,1
690373,2022-07-31,NaN,FR,2022-07-29,organic,organic,1659086493021-6361713490086303799,True,0,0.0,0.021129,1,1


In [ ]:
df['event_date'] = pd.to_datetime(df['event_date'])
df['install_date'] = pd.to_datetime(df['install_date'])
df.dtypes

event_date        datetime64[ns]
app_version               object
country_code              object
install_date      datetime64[ns]
media_source              object
campaign_name             object
appsflyer_id              object
is_organic                  bool
payments                   int64
revenue_usd              float64
ad_revenue_usd           float64
sessions                   int64
playtime                   int64
dtype: object

Расчет LTV будем осуществлять через cARPU - накопительный ARPU. Для того чтобы определить какие когорты можем считать созревшими для расчета попробуем рассчитать распределение по lifetime для пользователей.

In [ ]:
df['activity_day'] = (df['event_date'] - df['install_date']).dt.days
df

,event_date,app_version,country_code,install_date,media_source,campaign_name,appsflyer_id,is_organic,payments,revenue_usd,ad_revenue_usd,sessions,playtime,activity_day
0,2022-04-09,0.1.75.1,SG,2022-04-09,bytedanceglobal_int,EM_AND_SEA_Purchase_IT2_0604,1649548002569-6720420720726321605,False,0,0.0,0.000000,1,4,0
1,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649545238733-4551347935758619924,False,0,0.0,0.000000,1,2,0
2,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649540322226-5366969563919899948,False,0,0.0,0.000000,1,2,0
3,2022-04-09,0.1.75.1,UK,2022-04-07,bytedanceglobal_int,EM_AND_T1_Purchase_IT2_0604,1649337475698-617904100123526593,False,0,0.0,0.000000,1,1,2
4,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649544985557-9160306254813664580,False,0,0.0,0.000000,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690370,2022-07-31,NaN,PH,2022-07-28,organic,organic,1659003289354-9130797695840500796,True,0,0.0,0.002614,1,1,3
690371,2022-07-31,NaN,AE,2022-07-28,organic,organic,1659020308480-7169513513931602186,True,0,0.0,0.012164,1,1,3
690372,2022-07-31,NaN,DZ,2022-07-29,organic,organic,1659059933950-196412188438371439,True,0,0.0,0.000195,1,1,2
690373,2022-07-31,NaN,FR,2022-07-29,organic,organic,1659086493021-6361713490086303799,True,0,0.0,0.021129,1,1,2


In [ ]:
df_lt = df[df['install_date']>='2022-04-01'].groupby(['appsflyer_id','install_date'], as_index=False).agg({'activity_day':'max'}).rename(columns = {'activity_day':'lifetime'})
df_lt

,appsflyer_id,install_date,lifetime
0,1262310748796-5356552856195451152,2022-04-22,34
1,1483230136294-9150424549124065990,2022-06-26,0
2,1639063347507-7005281697189594540,2022-05-18,70
3,1639130247117-9144853101465218681,2022-07-21,0
4,1640546974307-4853705518684469702,2022-04-04,0
...,...,...,...
331198,1659617639772-3599033371225115285,2022-07-02,1
331199,1659786741876-641409135904582475,2022-07-30,0
331200,1659838019758-8805130209974056605,2022-07-31,0
331201,1659945407863-1685429817144280172,2022-07-31,0


In [ ]:
df_lt['cohort_month'] = df_lt['install_date'].dt.month
df_lt_gr = df_lt.groupby('cohort_month', as_index=False).agg({'lifetime':['mean','median']})
df_lt_gr.sort_values(by = 'cohort_month')

cohort_month  lifetime       
                    mean median
0            4  5.100924    0.0
1            5  4.213350    0.0
2            6  2.930373    0.0
3            7  1.621140    0.0

Какие выводы можем сделать по полученным результатам?
Посмотрим распределение по lifetime отдельно для апреля-мая.

In [ ]:
df_lt_distr = df_lt[df_lt['cohort_month']<=5].groupby(['cohort_month', 'lifetime'], as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'users'})
df_lt_distr

,cohort_month,lifetime,users
0,4,0,13968
1,4,1,1997
2,4,2,824
3,4,3,479
4,4,4,318
...,...,...,...
205,5,85,11
206,5,86,3
207,5,87,3
208,5,89,5


In [ ]:
fig3 = px.bar(df_lt_distr[df_lt_distr['cohort_month']==4], x='lifetime', y='users')
fig3.show()

Большинство пользователей имеют lifetime=0 (дней), однако они не дают увидеть распределение по lifetime для остальных пользователе - уберем их с графика

In [ ]:
fig4 = px.bar(df_lt_distr[df_lt_distr['cohort_month']==4], x='lifetime', y='users', range_x = [1,50])
fig4.show()

In [ ]:
fig5 = px.bar(df_lt_distr[df_lt_distr['cohort_month']==5], x='lifetime', y='users', range_x = [1,50])
fig5.show()

Для расчета LTV будет брать когорты с днем активности 30 дней и более. Для начала посчитаем общий LTV.

In [ ]:
df_ltv_all = df[(df['install_date']>='2022-04-01') & (df['install_date']<='2022-07-01')].agg({'appsflyer_id':'nunique', 'revenue_usd':'sum', 'ad_revenue_usd':'sum'})
df_ltv_all['ltv_gross'] = (df_ltv_all['revenue_usd'] + df_ltv_all['ad_revenue_usd'])/df_ltv_all['appsflyer_id']

df_ltv_all

appsflyer_id      201772.000000
revenue_usd        31619.591240
ad_revenue_usd      5628.563216
ltv_gross              0.184605
dtype: float64

Добавим сюда net LTV за вычетом 15% store cost

In [ ]:
df_ltv_all['ltv_net'] =  (df_ltv_all['revenue_usd']*0.85 + df_ltv_all['ad_revenue_usd'])/df_ltv_all['appsflyer_id']
df_ltv_all

appsflyer_id      201772.000000
revenue_usd        31619.591240
ad_revenue_usd      5628.563216
ltv_gross              0.184605
ltv_net                0.161099
dtype: float64

Таким образом, средний net LTV проекта = 0,16$

Посмотрим на отличия LTV для органики и платных пользователей

In [ ]:
df_ltv_channel =  df[(df['install_date']>='2022-04-01') & (df['install_date']<='2022-07-01')].groupby('is_organic', as_index = False).agg({'appsflyer_id':'nunique', 'revenue_usd':'sum', 'ad_revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'users'})
df_ltv_channel['ltv_gross'] = (df_ltv_channel['revenue_usd'] + df_ltv_channel['ad_revenue_usd'])/df_ltv_channel['users']
df_ltv_channel['ltv_net'] = (df_ltv_channel['revenue_usd']*0.85 + df_ltv_channel['ad_revenue_usd'])/df_ltv_channel['users']

df_ltv_channel

,is_organic,users,revenue_usd,ad_revenue_usd,ltv_gross,ltv_net
0,False,20726,3928.116579,916.140683,0.233729,0.205300
1,True,181140,27691.474661,4712.422533,0.178889,0.155958


Какие выводы можно сделать по полученным результатам?

Попробуем построить среднюю кривую LTV для первых 30 дней жизни пользователей.

In [ ]:
df_ltv_chart = df[(df['install_date']>='2022-04-01') & (df['install_date']<='2022-07-01')][['appsflyer_id', 'activity_day', 'revenue_usd', 'ad_revenue_usd']]
df_ltv_chart

,appsflyer_id,activity_day,revenue_usd,ad_revenue_usd
0,1649548002569-6720420720726321605,0,0.0,0.000000
1,1649545238733-4551347935758619924,0,0.0,0.000000
2,1649540322226-5366969563919899948,0,0.0,0.000000
3,1649337475698-617904100123526593,2,0.0,0.000000
4,1649544985557-9160306254813664580,0,0.0,0.000000
...,...,...,...,...
690349,1655810437409-9092904769387993777,40,0.0,0.013732
690350,1656088115144-7032414565871523030,37,0.0,0.003063
690351,1656225000202-5777117467800562417,35,0.0,0.000530
690352,1656259938831-6216107513489396529,35,0.0,0.001507


In [ ]:
df_days = pd.DataFrame({'days' : range(0,31)})
df_days

,days
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [ ]:
df_days['for_merge'] = 1
df_ltv_chart['for_merge'] = 1

df_ltv_chart = pd.merge(df_days, df_ltv_chart, on='for_merge')
df_ltv_chart = df_ltv_chart[df_ltv_chart['days']>=df_ltv_chart['activity_day']]

df_ltv_chart

,days,for_merge,appsflyer_id,activity_day,revenue_usd,ad_revenue_usd
0,0,1,1649548002569-6720420720726321605,0,0.0,0.000000
1,0,1,1649545238733-4551347935758619924,0,0.0,0.000000
2,0,1,1649540322226-5366969563919899948,0,0.0,0.000000
4,0,1,1649544985557-9160306254813664580,0,0.0,0.000000
5,0,1,1649543024686-2013762249241156998,0,0.0,0.000000
...,...,...,...,...,...,...
13787260,30,1,1656675870914-2979335606128628134,30,0.0,0.000845
13787288,30,1,1656585786681-4141387517245628942,30,0.0,0.001717
13787316,30,1,1656669176220-5527144952925588144,30,0.0,0.002075
13787329,30,1,1656699819244-1130979838401865670,30,0.0,0.002882


In [ ]:
df_ltv_chart = df_ltv_chart.groupby('days', as_index=False).agg({'appsflyer_id':'nunique', 'revenue_usd':'sum', 'ad_revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'users'})
df_ltv_chart['ltv_net'] = (df_ltv_chart['revenue_usd']*0.85 + df_ltv_chart['ad_revenue_usd'])/df_ltv_chart['users']

df_ltv_chart

,days,users,revenue_usd,ad_revenue_usd,ltv_net
0,0,197221,16728.075321,1582.948828,0.080122
1,1,199445,22335.979662,2305.437894,0.106751
2,2,200067,24255.916466,2763.221800,0.116865
3,3,200377,25525.662995,3114.761256,0.123824
4,4,200612,26237.919191,3391.574274,0.128077
5,5,200784,26846.786428,3607.298003,0.131619
6,6,200905,27344.504082,3802.917587,0.134620
7,7,200997,27840.051163,3966.003558,0.137465
8,8,201090,28197.181907,4104.732471,0.139601
9,9,201143,28428.044090,4221.492260,0.141120


In [ ]:
fig_ltv = px.line(df_ltv_chart, x= 'days', y = 'ltv_net')
fig_ltv.show()

# Домашнее задание

Выделите когорту пользователей у которой самой большой lifetime в игре. Выделите когорту пользователей у которой самой большой LTV в игре. Можно ли на основании имеющейся информации сделать рекомендации по закупке пользователей?

Когорта с самой большим LTV американский сектор.

Когорта пользователей у которой самой большой lifetime платящие американцы.

LTV в течении 30 днй увеличивется, отвал игроков в пределах нормы. Дополнительных рекомендации не требуется.

